In [ ]:
import os
import numpy as np
import joblib
from scipy.optimize import minimize

**Algorithm 2: Joint optimization of time offset and extrinsics**

Given:
  - Precomputed camera data:
    - `pipeline` with:
        - `images[frame_id]`
        - `edge_maps[frame_id]`
        - `dist_transforms[frame_id]`
    - `cam_times`  (array of camera timestamps [sec])
    - `frame_ids`  (array of frame_ids aligned with cam_times)
    - Camera intrinsics `K` (3×3)
  - LiDAR scans (list of dicts):
    - For each scan `s`:
        - `s["timestamp_sec"]`  (LiDAR time [sec])
        - `s["points"]`         (N×3 array of LiDAR-frame points)
  - Initial LiDAR→camera transform `T_CL_init` (4×4)

We want to find:
  - `params = [tau, rx, ry, rz, tx, ty, tz]`
  - that minimize:  `objective(params) = - mean_edge_score`.

---

**Step 1: Define helper transforms**

1.1. Convert small Euler angles to rotation matrix:
  - `R_delta = euler_to_R(rx, ry, rz)`.

1.2. Build correction transform:
  - `T_delta = make_T(R_delta, [tx, ty, tz])`.

1.3. Updated LiDAR→camera transform:
  - `T_CL = T_CL_init @ T_delta`.

1.4. Transform LiDAR points to camera frame:
  - For each point `p_L`:
      - `p_C = T_CL * [p_L; 1]` (homogeneous).
  - Collect into array `points_C`.

1.5. Project camera-frame points to pixels using intrinsics:
  - Keep only points with `Z > 0` (in front of camera).
  - Normalized coords:
      - `x_norm = X / Z`, `y_norm = Y / Z`.
  - Homogeneous pixel coords:
      - `[u; v; w] = K * [x_norm; y_norm; 1]`.
  - Pixels:
      - `u_px = u`, `v_px = v`.

---

**Step 2: Temporal association (LiDAR scan → camera frame)**

2.1. For a given LiDAR scan at time `t_L` and time offset `tau`:
  - Compute corresponding camera time:
      - `t_C = t_L + tau`.

2.2. Choose closest camera frame:
  - Find index `k = argmin |cam_times[k] - t_C|`.
  - Use `frame_id = frame_ids[k]`.

---

**Step 3: Per-scan edge alignment score**

3.1. For each LiDAR scan in the chosen time window:
  - Get LiDAR points `pts_L`.
  - Optionally subsample to `max_points_per_scan` for speed.

3.2. Transform to camera frame and project:
  - `uv = lidar_points_to_pixels(pts_L_sub, T_CL, K)`.

3.3. Query CameraPipeline distance transform:
  - Use `pipeline.score_projection(frame_id, uv)`.
  - This:
      - Discards points outside the image.
      - Looks up distance `d` to nearest edge for each projected point.
      - Converts distances to scores via `exp(-d^2 / (2 * sigma^2))`.
      - Sums all scores ⇒ `scan_score`.

---

**Step 4: Define global objective function**

4.1. For a given `params = [tau, rx, ry, rz, tx, ty, tz]`:

  - Initialize:
      - `total_score = 0`
      - `valid_count = 0`.

  - For each chosen LiDAR scan index `i`:
      1. Compute `scan_score_i` as in Step 3.
      2. If `uv` is non-empty:
          - `total_score += scan_score_i`
          - `valid_count += 1`.

4.2. If `valid_count == 0`:
  - Return `objective = 0.0` (bad alignment).

4.3. Else:
  - `mean_score = total_score / valid_count`.
  - Return:
      - `objective(params) = -mean_score`
      - (negative because optimizers *minimize*).

---

**Step 5: Optimization strategy**

- **Stage 1: tau-only optimization**
  1. Fix `rx, ry, rz, tx, ty, tz = 0`.
  2. Optimize only `tau`:
     - `minimize(tau_only_objective, x0=[0.0])`.
  3. Get best time offset `tau*`.

- **Stage 2: full joint refinement (optional)**
  1. Start from:
     - `x0_full = [tau*, 0, 0, 0, 0, 0, 0]`.
  2. Optimize all 7 parameters:
     - `minimize(joint_objective, x0_full, method="Powell")`.
  3. Final result:
     - `tau*`, `R_delta*`, `t_delta*`.
  4. Updated extrinsics:
     - `T_CL_final = T_CL_init @ T_delta*`.

Result:
  - A time offset and LiDAR→camera transform that maximize LiDAR–camera edge alignment across the calibration time window.


In [ ]:
# ========== 1. Small Euler -> rotation matrix ==========
def euler_to_R(rx, ry, rz):
    cx, sx = np.cos(rx), np.sin(rx)
    cy, sy = np.cos(ry), np.sin(ry)
    cz, sz = np.cos(rz), np.sin(rz)

    Rx = np.array([
        [1, 0, 0],
        [0, cx, -sx],
        [0, sx,  cx]
    ])

    Ry = np.array([
        [ cy, 0, sy],
        [  0, 1,  0],
        [-sy, 0, cy]
    ])

    Rz = np.array([
        [cz, -sz, 0],
        [sz,  cz, 0],
        [ 0,   0, 1]
    ])

    return Rz @ Ry @ Rx


# ========== 2. Build 4x4 transform from R, t ==========
def make_T(R, t):
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3]  = t.reshape(3,)
    return T


# ========== 3. LiDAR -> Camera frame ==========
def transform_lidar_to_cam(points_L, T_CL):
    if points_L.shape[0] == 0:
        return np.zeros((0, 3), dtype=np.float32)

    N = points_L.shape[0]
    pts_L_h = np.hstack([points_L, np.ones((N, 1))])  # (N,4)
    pts_C_h = (T_CL @ pts_L_h.T).T                    # (N,4)
    return pts_C_h[:, :3]


# ========== 4. Camera frame -> pixels using K ==========
def project_cam_points_to_pixels(points_C, K):
    if points_C.shape[0] == 0:
        return np.zeros((0, 2), dtype=np.float32)

    X = points_C[:, 0]
    Y = points_C[:, 1]
    Z = points_C[:, 2]

    # Keep only points in front of camera
    mask = Z > 0
    X, Y, Z = X[mask], Y[mask], Z[mask]
    if X.size == 0:
        return np.zeros((0, 2), dtype=np.float32)

    x_norm = X / Z
    y_norm = Y / Z

    pts_norm = np.vstack([x_norm, y_norm, np.ones_like(x_norm)])  # (3,M)
    uv_h = K @ pts_norm                                            # (3,M)

    u = uv_h[0, :]
    v = uv_h[1, :]
    uv = np.stack([u, v], axis=1).astype(np.float32)               # (M,2)
    return uv


# ========== 5. LiDAR points -> pixel coords ==========
def lidar_points_to_pixels(points_L, T_CL, K):
    points_C = transform_lidar_to_cam(points_L, T_CL)
    uv = project_cam_points_to_pixels(points_C, K)
    return uv


# ========== 6. Choose closest camera frame for a given time ==========
def closest_frame_id(cam_times, frame_ids, t_C):
    idx = int(np.argmin(np.abs(cam_times - t_C)))
    return int(frame_ids[idx])


# ========== 7. Build joint optimization objective ==========
def joint_objective(params,
                    pipeline,
                    lidar_scans,
                    cam_times,
                    frame_ids,
                    K,
                    T_CL_init,
                    scan_indices=None,
                    max_points_per_scan=5000):

    cam_times = np.asarray(cam_times)
    frame_ids = np.asarray(frame_ids)

    if scan_indices is None:
        scan_indices = np.arange(len(lidar_scans))
    else:
        scan_indices = np.asarray(scan_indices)

    tau = params[0]
    rx, ry, rz = params[1:4]
    tx, ty, tz = params[4:7]

    R_delta = euler_to_R(rx, ry, rz)
    t_delta = np.array([tx, ty, tz], dtype=float)
    T_delta = make_T(R_delta, t_delta)
    T_CL = T_CL_init @ T_delta

    total_score = 0.0
    valid_count = 0

    for idx in scan_indices:
        scan = lidar_scans[idx]
        t_L   = scan["timestamp_sec"]
        pts_L = scan["points"]

        if max_points_per_scan is not None and pts_L.shape[0] > max_points_per_scan:
            choice = np.random.choice(pts_L.shape[0], max_points_per_scan, replace=False)
            pts_L_sub = pts_L[choice]
        else:
            pts_L_sub = pts_L

        t_C = t_L + tau
        frame_id = closest_frame_id(cam_times, frame_ids, t_C)

        uv = lidar_points_to_pixels(pts_L_sub, T_CL, K)
        if uv.shape[0] == 0:
            continue

        s = pipeline.score_projection(frame_id, uv)
        total_score += s
        valid_count += 1

    if valid_count == 0:
        return 0.0

    mean_score = total_score / valid_count
    return -mean_score




In [ ]:
# ======== helper functions ========
# euler_to_R
# make_T
# transform_lidar_to_cam
# project_cam_points_to_pixels
# lidar_points_to_pixels
# closest_frame_id
# joint_objective   

# ---------- tau-only wrapper ----------
def tau_only_objective(tau_vec,
                       pipeline,
                       lidar_scans,
                       cam_times,
                       frame_ids,
                       K,
                       T_CL_init,
                       scan_indices=None,
                       max_points_per_scan=5000):
    tau = float(tau_vec[0])

    params = np.zeros(7, dtype=float)
    params[0] = tau  # only tau is free, others = 0

    return joint_objective(
        params,
        pipeline=pipeline,
        lidar_scans=lidar_scans,
        cam_times=cam_times,
        frame_ids=frame_ids,
        K=K,
        T_CL_init=T_CL_init,
        scan_indices=scan_indices,
        max_points_per_scan=max_points_per_scan,
    )

# ============ LOAD ALL PRECOMPUTED DATA ============

calib_dir = "/home/aryaman/Forsyth_Data/calib_cache"

pipeline  = joblib.load(os.path.join(calib_dir, "camera_pipeline.pkl"))
cam_times = np.load(os.path.join(calib_dir, "cam_times.npy"))
frame_ids = np.load(os.path.join(calib_dir, "frame_ids.npy"))

fx = 1501.9374712879626
fy = 1498.8879775647906
cx = 566.5690420612353
cy = 537.1294320963829

K = np.array([
    [fx, 0.0, cx],
    [0.0, fy, cy],
    [0.0, 0.0, 1.0]
], dtype=float)

print("K =\n", K)

lidar_scans_path = "/home/aryaman/Forsyth_Data/lidar_scans.pkl"
lidar_scans = joblib.load(lidar_scans_path)

T_CL_init = np.array([
    [0.99919851,  0.04002921,  0.00000000,   0.15],
    [0.00000000,  0.00000000, -1.00000000,  -0.2815789473684212],
    [-0.04002921, 0.99919851,  0.00000000,  -0.13157894736842124],
    [0.0,         0.0,         0.0,          1.0]
], dtype=float)

print("T_CL_init =\n", T_CL_init)

# ============ STAGE 1: tau-only optimization ============

x0_tau = np.array([0.0])  # initial tau

print("\n--- Stage 1: tau-only optimization ---")
res_tau = minimize(
    tau_only_objective,
    x0_tau,
    args=(pipeline, lidar_scans, cam_times, frame_ids, K, T_CL_init, None, 5000),
    method="Powell"
)
best_tau = float(res_tau.x[0])
print("Optimized tau =", best_tau)
print("Stage 1 objective =", res_tau.fun)

# ============ STAGE 2: full 7D optimization (optional) ============

# If you want to *only* calibrate tau and keep R,t fixed, you can stop here.

# Otherwise, refine all 7 starting from the tau we just found:
x0_full = np.zeros(7)
x0_full[0] = best_tau

print("\n--- Stage 2: full [tau, R, t] optimization ---")
res_full = minimize(
    joint_objective,
    x0_full,
    args=(pipeline, lidar_scans, cam_times, frame_ids, K, T_CL_init, None, 5000),
    method="Powell"
)

print("\n--- Optimization Complete ---")
print("Optimized params =", res_full.x)
print("Final objective   =", res_full.fun)
